In [2]:
from main2 import _scrape_conversion_table, _get_tract_geoms_state, _calc_pct_overlap, create_year_conversion_map
import pandas as pd
import numpy as np
import requests
import os
import pygris

In [3]:
DATA_DIR = 'data' 
CONVERSION_PATH = os.path.join(DATA_DIR, 'tract_conversion_table_2010-2010_raw.csv') 
# GEOMS_PATH = os.path.join(DATA_DIR, 'tract_conversion_table_2010-2010_geometries.json')
OUTPUT_PATH = os.path.join(DATA_DIR, 'tract_conversion_table_2010-2010_processed-overlaps.csv')
OVERLAP_PRECISION = 3 # how many decimals you want to calculate the overlap with


In [4]:
## scrape / load conversion table 
_scrape_conversion_table(CONVERSION_PATH)
conversion_table = pd.read_csv(CONVERSION_PATH, sep='|', dtype={'GEOID_TRACT_10':str, 'GEOID_TRACT_20':str})
conversion_table = conversion_table[['STATENAME','GEOID_TRACT_10', 'GEOID_TRACT_20']]

## download geometries   
for state in conversion_table['STATENAME'].unique():
    geoms_10, geoms_20 = _get_tract_geoms_state(state=state, year=2010), _get_tract_geoms_state(state=state, year=2020)

    # Join to the conversion table 
    conversion_table = conversion_table\
        .merge(geoms_10, how='left', left_on='GEOID_TRACT_10', right_index=True)\
        .merge(geoms_20, how='left', left_on='GEOID_TRACT_20', right_index=True)

    # Calculate percent overlap bidirectionally 
    conversion_table[['pct_overlap_10-to-20', 'pct_overlap_20-to-10']] = conversion_table\
        .apply(lambda row: pd.Series([_calc_pct_overlap(row['geometry_10'], row['geometry_20']), 
                                        _calc_pct_overlap(row['geometry_20'], row['geometry_10'])]), axis=1)
    
    conversion_table


2023-11-29 18:49:25,491 - logger - INFO - data/tract_conversion_table_2010-2010_raw.csv already exists. Skipping scrape.


Using FIPS code '01' for input 'Alabama'
Using FIPS code '01' for input 'Alabama'


In [7]:
_scrape_conversion_table(CONVERSION_PATH)
conversion_table = pd.read_csv(CONVERSION_PATH, sep='|', dtype={'GEOID_TRACT_10':str, 'GEOID_TRACT_20':str})
conversion_table = conversion_table[['STATENAME','GEOID_TRACT_10', 'GEOID_TRACT_20']]

## download geometries (TO-DO: change _get_tract_geoms_state() to return a list of dicts and then convert finished to dataframe after loop. Compare speed vs appending dataframes and concatenating like here)
geoms_10, geoms_20 = [], []  
for state in conversion_table['STATENAME'].unique():
    geoms_10_state, geoms_20_state = _get_tract_geoms_state(state=state, year=2010), _get_tract_geoms_state(state=state, year=2020)
    geoms_10.append(geoms_10_state)
    geoms_20.append(geoms_20_state)
geoms_10 = pd.concat(geoms_10)
geoms_20 = pd.concat(geoms_20)


# Join to the conversion table 
conversion_table = conversion_table\
    .merge(geoms_10, how='left', left_on='GEOID_TRACT_10', right_index=True)\
    .merge(geoms_20, how='left', left_on='GEOID_TRACT_20', right_index=True)

# Calculate percent overlap bidirectionally 
conversion_table[['pct_overlap_10-to-20', 'pct_overlap_20-to-10']] = conversion_table\
    .apply(lambda row: pd.Series([_calc_pct_overlap(row['geometry_10'], row['geometry_20']), 
                                    _calc_pct_overlap(row['geometry_20'], row['geometry_10'])]), axis=1)
    
# Create maps from one year to the other
map_10_to_20 = create_year_conversion_map(conversion_table, start_year=2010, end_year=2020, dec_to_round=OVERLAP_PRECISION)
map_20_to_10 = create_year_conversion_map(conversion_table, start_year=2020, end_year=2010, dec_to_round=OVERLAP_PRECISION)



2023-11-29 18:50:28,457 - logger - INFO - data/tract_conversion_table_2010-2010_raw.csv already exists. Skipping scrape.


Using FIPS code '01' for input 'Alabama'
Using FIPS code '01' for input 'Alabama'


NameError: name 'OVERLAP_PRECISION' is not defined

In [5]:
pivot_conversion_table(conversion_table, start_year=2010, end_year=2020)# conversion_table.groupby(['GEOID_TRACT_20'])['pct_overlap_20-to-10'].max().reset_index()

NameError: name 'pivot_conversion_table' is not defined

In [90]:
df = conversion_table
start_year, end_year = 2010, 2020
dec_to_round = 4

start_year_abrev, end_year_abrev = str(start_year)[-2:], str(end_year)[-2:]
start_geoid, end_geoid = f"GEOID_TRACT_{start_year_abrev}", f"GEOID_TRACT_{end_year_abrev}"
pct_overlap_col = f'pct_overlap_{start_year_abrev}-to-{end_year_abrev}'

map_series = df.groupby(['STATENAME', start_geoid])[[end_geoid, pct_overlap_col]] \
    .agg(list)\
    .apply(lambda row: {x[0]:np.round((x[1]), dec_to_round) for x in zip(row[end_geoid], row[pct_overlap_col])}, axis=1)\
    .reset_index()\
    .rename({0:f'{end_geoid}_overlap'}, axis=1) \
    .to_dict(orient='records')
    

map_series

# max_len = df_widened[end_geoid].agg(len).max()
# for i in range(1,max_len+1): 
#     df_widened[f'{end_geoid}-{i}'] = df_widened[f'{end_geoid}'].apply(lambda x: x[i] if i < len(x) else None)
# df_widened.drop([end_geoid], axis=1, inplace=True)
# df_widened

# max_len
#     df_widened[f'{end_geoid}-{i}'] = df_widened[f'{end_geoid}'].apply(lambda x: x[i] if i < len(x) else None)
# df_widened.drop([end_geoid], axis=1, inplace=True)

# df_widened
# # max_len = df_widened[f'overlapping_geoids_{end_year_abrev}'].len().max()

[{'STATENAME': 'Alabama',
  'GEOID_TRACT_10': '01001020100',
  'GEOID_TRACT_20_overlap': {'01001020100': 99.9093, '01001020803': 0.0011}},
 {'STATENAME': 'Alabama',
  'GEOID_TRACT_10': '01001020200',
  'GEOID_TRACT_20_overlap': {'01001020100': 0.0493, '01001020200': 99.96}},
 {'STATENAME': 'Alabama',
  'GEOID_TRACT_10': '01001020300',
  'GEOID_TRACT_20_overlap': {'01001020300': 100.0}},
 {'STATENAME': 'Alabama',
  'GEOID_TRACT_10': '01001020400',
  'GEOID_TRACT_20_overlap': {'01001020400': 99.9907}},
 {'STATENAME': 'Alabama',
  'GEOID_TRACT_10': '01001020500',
  'GEOID_TRACT_20_overlap': {'01001020501': 99.9999,
   '01001020502': 99.9999,
   '01001020503': 99.7822}},
 {'STATENAME': 'Alabama',
  'GEOID_TRACT_10': '01001020600',
  'GEOID_TRACT_20_overlap': {'01001020600': 99.7403}},
 {'STATENAME': 'Alabama',
  'GEOID_TRACT_10': '01001020700',
  'GEOID_TRACT_20_overlap': {'01001020700': 99.9996}},
 {'STATENAME': 'Alabama',
  'GEOID_TRACT_10': '01001020801',
  'GEOID_TRACT_20_overlap': {'0

In [91]:
OVERLAP_PRECISION = 3
map_10_to_20 = create_year_conversion_map(conversion_table, start_year=2010, end_year=2020, dec_to_round=OVERLAP_PRECISION)
map_20_to_10 = create_year_conversion_map(conversion_table, start_year=2020, end_year=2010, dec_to_round=OVERLAP_PRECISION)


NameError: name 'create_year_conversion_map' is not defined

In [ ]:
df = conversion_table

df.groupby(['GEOID_TRACT_10'])['GEOID_TRACT_20'] \
        .agg([list])

,list
GEOID_TRACT_10,
01001020100,"[01001020100, 01001020803]"
01001020200,"[01001020100, 01001020200]"
01001020300,[01001020300]
01001020400,[01001020400]
01001020500,"[01001020501, 01001020502, 01001020503]"
...,...
01133965503,[01133965503]
01133965600,"[01133965501, 01133965502, 01133965601, 011339..."
01133965700,[01133965700]


In [ ]:
conversion_table.dtypes

STATENAME         object
GEOID_TRACT_10     int64
GEOID_TRACT_20     int64
dtype: object

In [ ]:
conversion_table.columns

Index(['STATENAME', 'GEOID_TRACT_10', 'GEOID_TRACT_20'], dtype='object')

In [ ]:
conversion_table.groupby('GEOID_TRACT_10')['GEOID_TRACT_20'].agg(list).reset_index()
max_len = df['GEOID_TRACT_20'].apply(len).max()
for i in range(max_len): 
    df[f'GEOID_TRACT_20-{i}'] =  df['GEOID_TRACT_20'].apply(lambda x: x[i] if i < len(x) else None)
df.drop(['GEOID_TRACT_20'], axis=1, inplace=True)

,GEOID_TRACT_10,GEOID_TRACT_20
0,01001020100,"[01001020100, 01001020803]"
1,01001020200,"[01001020100, 01001020200]"
2,01001020300,[01001020300]
3,01001020400,[01001020400]
4,01001020500,"[01001020501, 01001020502, 01001020503]"
...,...,...
1176,01133965503,[01133965503]
1177,01133965600,"[01133965501, 01133965502, 01133965601, 011339..."
1178,01133965700,[01133965700]
1179,01133965800,[01133965800]


In [ ]:
df = pd.DataFrame(table)
df_2010_to_2020 = df.groupby('GEOID_TRACT_10')['GEOID_TRACT_20'].agg(list).reset_index().to_dict(orient='records')
df_2010_to_2020

[{'GEOID_TRACT_10': '10001040100', 'GEOID_TRACT_20': ['10001040100']},
 {'GEOID_TRACT_10': '10001040201',
  'GEOID_TRACT_20': ['10001040201', '10001040204', '10003016901']},
 {'GEOID_TRACT_10': '10001040202',
  'GEOID_TRACT_20': ['10001040204', '10001040205', '10001040206']},
 {'GEOID_TRACT_10': '10001040203', 'GEOID_TRACT_20': ['10001040203']},
 {'GEOID_TRACT_10': '10001040501', 'GEOID_TRACT_20': ['10001040501']},
 {'GEOID_TRACT_10': '10001040502', 'GEOID_TRACT_20': ['10001040502']},
 {'GEOID_TRACT_10': '10001040700',
  'GEOID_TRACT_20': ['10001040700', '10001043300']},
 {'GEOID_TRACT_10': '10001040900', 'GEOID_TRACT_20': ['10001040900']},
 {'GEOID_TRACT_10': '10001041000',
  'GEOID_TRACT_20': ['10001041000', '10001041200', '10001980000']},
 {'GEOID_TRACT_10': '10001041100',
  'GEOID_TRACT_20': ['10001041101',
   '10001041200',
   '10001042203',
   '10001043202',
   '10001980000']},
 {'GEOID_TRACT_10': '10001041200',
  'GEOID_TRACT_20': ['10001041000', '10001041101', '10001041200']},


In [ ]:
def get_conversion_table():
    # get conversion  table from the census website
    conversion_table = requests.get(
        'https://www2.census.gov/geo/docs/maps-data/data/rel2020/tract/tab20_tract20_tract10_st10.txt')
    lines = conversion_table.text.splitlines()
    # fix typo
    lines[0] = lines[0].replace('ï»¿', '')

    census_tract_conversion_list = []
    headers = lines.pop(0).split('|')
    for line in lines:
        my_dict = {}
        line = line.split('|')
        for cat in headers:
            index = headers.index(cat)
            my_dict[cat] = line[index]
        census_tract_conversion_list.append(my_dict)
    return census_tract_conversion_list


In [ ]:
df = pd.DataFrame(get_conversion_table()).groupby('GEOID_TRACT_10')['GEOID_TRACT_20'].agg(list).reset_index()
max_len = df['GEOID_TRACT_20'].apply(len).max()
for i in range(max_len): 
    df[f'GEOID_TRACT_20-{i}'] =  df['GEOID_TRACT_20'].apply(lambda x: x[i] if i < len(x) else None)
df.drop(['GEOID_TRACT_20'], axis=1, inplace=True)

display(df)

,GEOID_TRACT_10,GEOID_TRACT_20-0,GEOID_TRACT_20-1,GEOID_TRACT_20-2,GEOID_TRACT_20-3,GEOID_TRACT_20-4,GEOID_TRACT_20-5,GEOID_TRACT_20-6
0,10001040100,10001040100,None,None,None,None,None,None
1,10001040201,10001040201,10001040204,10003016901,None,None,None,None
2,10001040202,10001040204,10001040205,10001040206,None,None,None,None
3,10001040203,10001040203,None,None,None,None,None,None
4,10001040501,10001040501,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...
213,10005051702,10005051702,None,None,None,None,None,None
214,10005051801,10005051801,None,None,None,None,None,None
215,10005051802,10005051802,None,None,None,None,None,None
216,10005051900,10005051900,None,None,None,None,None,None


In [ ]:
df.

In [ ]:
conversion_table = pd.read_csv('data/tract_conversion_table_2010-2010_raw.csv', sep='|')

In [ ]:
conversion_table.columns

Index(['STATENAME', 'OID_TRACT_20', 'GEOID_TRACT_20', 'NAMELSAD_TRACT_20',
       'AREALAND_TRACT_20', 'AREAWATER_TRACT_20', 'MTFCC_TRACT_20',
       'FUNCSTAT_TRACT_20', 'OID_TRACT_10', 'GEOID_TRACT_10',
       'NAMELSAD_TRACT_10', 'AREALAND_TRACT_10', 'AREAWATER_TRACT_10',
       'MTFCC_TRACT_10', 'FUNCSTAT_TRACT_10', 'AREALAND_PART',
       'AREAWATER_PARTAlabama', '20790540092527', '01001020100',
       'Census Tract 201', '9825304', '28435', 'G5020', 'S', '20740540092527',
       '01001020100.1', 'Census Tract 201.1', '9827271', '28435.1', 'G5020.1',
       'S.1', '9820448', '28435.2'],
      dtype='object')